In [1]:
using Crystalline, MPBUtils, JLD2, LinearAlgebra, StaticArrays, SymmetryBases, PyPlot, DelimitedFiles, PrettyTables

In [16]:
sg_wyckoffs = [filter(x -> x.mult == 1, wyckoffs(i, 2)) for i in (2, 6, (9:17)...)]
sg_wyckoffs[3] = wyckoffs(9, 2)[end-1:end];
sg_wyckoffs[6] = [wyckoffs(12, 2)[end]];

In [69]:
sg_wyckoffs_dict = Dict(([2, 6, 9:17...])[i] => sg_wyckoffs[i] for i in 1:11)
sg_wyckoffs_dict_sorted = sort(collect(sg_wyckoffs_dict), by = x -> x[1] );

In [74]:
wyckoffs(2,2)

5-element Vector{WyckoffPosition{2}}:
 2e: [α, β]
 1d: [1/2, 1/2]
 1c: [1/2, 0]
 1b: [0, 1/2]
 1a: [0, 0]

In [76]:
sg_wyckoffs_dict

Dict{Int64, Vector{WyckoffPosition{2}}} with 11 entries:
  16 => [1a: [0, 0]]
  12 => [2a: [0, 0]]
  17 => [1a: [0, 0]]
  6  => [1d: [1/2, 1/2], 1c: [1/2, 0], 1b: [0, 1/2], 1a: [0, 0]]
  11 => [1b: [1/2, 1/2], 1a: [0, 0]]
  9  => [2b: [0, 1/2], 2a: [0, 0]]
  14 => [1c: [2/3, 1/3], 1b: [1/3, 2/3], 1a: [0, 0]]
  13 => [1c: [2/3, 1/3], 1b: [1/3, 2/3], 1a: [0, 0]]
  15 => [1a: [0, 0]]
  2  => [1d: [1/2, 1/2], 1c: [1/2, 0], 1b: [0, 1/2], 1a: [0, 0]]
  10 => [1b: [1/2, 1/2], 1a: [0, 0]]

In [2]:
pretty_table(["0,0" "1/2,0" "0,1/2" "1/2,1/2" ; "0,0" "1/2,0" "0,1/2" "1/2,1/2"; "0,0" "1/2,0" "N/A" "N/A" ], header=["Center 1",
    "Center 2", "Center 3", "Center 4"], row_names=["p2", "p2mm", "c2mm"], title="C2 Symmetry Centering Choices")
pretty_table(["0,0" "2/3,1/3" "1/3,2/3"; "0,0" "2/3,1/3" "1/3,2/3"; "0,0" "1/3,2/3" "N/A" ], header=["Center 1",
    "Center 2", "Center 3"], row_names=["p3", "p3m1", "p31m"], title="C3 Symmetry Centering Choices")
pretty_table(["0,0" "1/2,1/2"; "0,0" "1/2,1/2"; "0,0" "N/A" ], header=["Center 1",
    "Center 2"], row_names=["p4", "p4mm", "p4gm"], title="C4 Symmetry Centering Choices")

C2 Symmetry Centering Choices
┌──────┬──────────┬──────────┬──────────┬──────────┐
│      │ Center 1 │ Center 2 │ Center 3 │ Center 4 │
├──────┼──────────┼──────────┼──────────┼──────────┤
│   p2 │      0,0 │    1/2,0 │    0,1/2 │  1/2,1/2 │
│ p2mm │      0,0 │    1/2,0 │    0,1/2 │  1/2,1/2 │
│ c2mm │      0,0 │    1/2,0 │      N/A │      N/A │
└──────┴──────────┴──────────┴──────────┴──────────┘
C3 Symmetry Centering Choices
┌──────┬──────────┬──────────┬──────────┐
│      │ Center 1 │ Center 2 │ Center 3 │
├──────┼──────────┼──────────┼──────────┤
│   p3 │      0,0 │  2/3,1/3 │  1/3,2/3 │
│ p3m1 │      0,0 │  2/3,1/3 │  1/3,2/3 │
│ p31m │      0,0 │  1/3,2/3 │      N/A │
└──────┴──────────┴──────────┴──────────┘
C4 Symmetry Centering Choices
┌──────┬──────────┬──────────┐
│      │ Center 1 │ Center 2 │
├──────┼──────────┼──────────┤
│   p4 │      0,0 │  1/2,1/2 │
│ p4mm │      0,0 │  1/2,1/2 │
│ p4gm │      0,0 │      N/A │
└──────┴──────────┴──────────┘


In [46]:
lgirreps(6, 2)[]

Dict{String, Vector{LGIrrep{2}}} with 9 entries:
  "Y" => [LGIrrep{2}("Y₁", [1, 2, m₁₀, m₀₁] ([0, 1/2]), Matrix{ComplexF…
  "Δ" => [LGIrrep{2}("Δ₁", [1, m₁₀] ([0, α]), Matrix{ComplexF64}[[1.0+0…
  "S" => [LGIrrep{2}("S₁", [1, 2, m₁₀, m₀₁] ([1/2, 1/2]), Matrix{Comple…
  "X" => [LGIrrep{2}("X₁", [1, 2, m₁₀, m₀₁] ([1/2, 0]), Matrix{ComplexF…
  "C" => [LGIrrep{2}("C₁", [1, m₀₁] ([α, 1/2]), Matrix{ComplexF64}[[1.0…
  "Σ" => [LGIrrep{2}("Σ₁", [1, m₀₁] ([α, 0]), Matrix{ComplexF64}[[1.0+0…
  "Γ" => [LGIrrep{2}("Γ₁", [1, 2, m₁₀, m₀₁] ([0, 0]), Matrix{ComplexF64…
  "D" => [LGIrrep{2}("D₁", [1, m₁₀] ([1/2, α]), Matrix{ComplexF64}[[1.0…
  "Ω" => [LGIrrep{2}("Ω₁", [1] ([α, β]), Matrix{ComplexF64}[[1.0+0.0im;…

In [47]:
pretty_table(["B1" "Y1" "A1" "Γ1" "1/4(-[B1]-[Y1]+[A1]+[Γ1])" "([Y1]+[A1], [B1]+[A1])"; 
    "B1" "Y2" "A2" "Γ1" "1/4(-[B1]-[Y2]+[A2]+[Γ1])" "([Y2]+[A2], [B1]+[A2])";
    "B2" "Y1" "A2" "Γ1" "1/4(-[B2]-[Y1]+[A2]+[Γ1])" "([Y1]+[A2], [B2]+[A2])";
    "B2" "Y2" "A1" "Γ1" "1/4(-[B2]-[Y2]+[A1]+[Γ1])" "([Y2]+[A1], [B2]+[A1])"], 
    header=["B (λ₂ = 1)", "Y Irrep (λ₂ = 1)", "A Irrep (λ₂ = 1)", "Γ1 (λ₂ = 1)", "Corner Charge", "Polarization"], 
row_names = ["0,0", "0,1/2", "1/2,0", "1/2,1/2"], title="p2 Irreps", backend=:text, crop=:both)
    

pretty_table(["(X1, X2)" "(Y1, Y2)" "(S1, S2)" "(Γ1, Γ2)" "1/4(-[X1]-[X2]-[Y1]-[Y2]+[S1]+[S2]+[Γ1]+[Γ2])" "1/2([Y1]+[Y2]+[S1]+[S2], [X1]+[X2]+[S1]+[S2])";
        "(X1, X2)" "(Y3, Y4)" "(S3, S4)" "(Γ1, Γ2)" "1/4(-[X1]-[X2]-[Y3]-[Y4]+[S3]+[S4]+[Γ1]+[Γ2])" "1/2([Y3]+[Y4]+[S3]+[S4], [X1]+[X2]+[S3]+[S4])";
        "(X3, X4)" "(Y1, Y2)" "(S3, S4)" "(Γ1, Γ2)" "1/4(-[X3]-[X4]-[Y1]-[Y2]+[S3]+[S4]+[Γ1]+[Γ2])" "1/2([Y1]+[Y2]+[S3]+[S4], [X3]+[X4]+[S3]+[S4])";
        "(X3, X4)" "(Y3, Y4)" "(S1, S2)" "(Γ1, Γ2)" "1/4(-[X3]-[X4]-[Y3]-[Y4]+[S1]+[S2]+[Γ1]+[Γ2])" "1/2([Y3]+[Y4]+[S1]+[S2], [X3]+[X4]+[S1]+[S2])"] ,
    header=["X Irrep (λ₂ = 1)", "Y Irrep (λ₂ = 1)", "S Irrep (λ₂ = 1)", "Γ Irrep (λ₂ = 1)",  "Corner Charge", "Polarization"], 
row_names = ["0,0", "0,1/2", "1/2,0", "1/2,1/2"], title="p2mm Irreps", crop=:both)


pretty_table([ "(Y1, Y2)" "S1"; "(Y1, Y2)" "S2"], header=["Y Irrep", "S Irrep"],

row_names = ["0,0", "0,1/2"], title="c2mm Irreps")

pretty_table([ "X1" "M1" "(M3M4)"; "X2" "M2" "M3M4"], header=["X Irrep", "M Irrep", "M Irrep"],

row_names = ["0,0", "1/2,1/2"], title="c4 Irreps")

pretty_table([ "(X1, X2)" "(M1, M4)" "M5" ; "(X3, X4)" "(M2, M3)" "M5"], header=["X Irrep", "M Irrep", "M Irrep"],

row_names = ["0,0", "1/2,1/2"], title="c4mm Irreps")

pretty_table([ "K1" "K2"; "K2" "K3"; "K3" "K1"], header=["K Irrep", "K Irrep"],

row_names = ["0,0", "1/3,2/3", "2/3,1/3"], title="p3 Irreps")

pretty_table([ "K1" "K2"; "K2" "K3"; "K3" "K1"], header=["K Irrep", "K Irrep"],

row_names = ["0,0", "1/3,2/3", "2/3,1/3"], title="p3mm Irreps")

p2 Irreps
┌─────────┬────────────┬──────────────────┬──────────────────┬─────────────┬────
│         │ B (λ₂ = 1) │ Y Irrep (λ₂ = 1) │ A Irrep (λ₂ = 1) │ Γ1 (λ₂ = 1) │   ⋯
├─────────┼────────────┼──────────────────┼──────────────────┼─────────────┼────
│     0,0 │         B1 │               Y1 │               A1 │          Γ1 │ 1 ⋯
│   0,1/2 │         B1 │               Y2 │               A2 │          Γ1 │ 1 ⋯
│   1/2,0 │         B2 │               Y1 │               A2 │          Γ1 │ 1 ⋯
│ 1/2,1/2 │         B2 │               Y2 │               A1 │          Γ1 │ 1 ⋯
└─────────┴────────────┴──────────────────┴──────────────────┴─────────────┴────
                                                               2 columns omitted
p2mm Irreps
┌─────────┬──────────────────┬──────────────────┬──────────────────┬────────────
│         │ X Irrep (λ₂ = 1) │ Y Irrep (λ₂ = 1) │ S Irrep (λ₂ = 1) │ Γ Irrep ( ⋯
├─────────┼──────────────────┼──────────────────┼──────────────────┼────────────
│     

In [15]:
for sg in [2, 6, 9:14...]
    origin_line = filter(x -> contains(x, "origin"), readlines("./symeigs/shiftedlattices/input/dim2-sg$sg-2-res64-tm.sh"))
    println(origin_line)
    #println(String.(split(first(origin_line))))
end

["origin=(vector3 0 0.5)"]
["origin=(vector3 0 0.5)"]
["origin=(vector3 -0.5 0.5)"]
["origin=(vector3 0.5 0.5)"]
["origin=(vector3 0.5 0.5)"]
["origin=(vector3 0.5 0.5)"]
["origin=(vector3 0.33333333333 0.6666666666)"]
["origin=(vector3 0.33333333333 0.66666666666)"]
